In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [5]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [6]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [7]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [8]:
data.to_excel('1.xlsx')

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [9]:
pd.read_sql(
    """
    SELECT 
    count(*) - count(CRIM) as CRIM,
    count(*) - count(ZN) as ZN,
    count(*) - count(INDUS) as INDUS,
    count(*) - count(CHAS) as CHAS,
    count(*) - count(NOX) as NOX
    FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [10]:
pd.read_sql(
    """
    SELECT 
    count(DISTINCT CRIM) as CRIM,
    count(DISTINCT ZN) as ZN,
    count(DISTINCT INDUS) as INDUS,
    count(DISTINCT CHAS) as CHAS,
    count(DISTINCT NOX) as NOX
    FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [75]:
pd.read_sql(
    """
    SELECT 
    
    CASE
    WHEN (SELECT AVG(CRIM)
    FROM (SELECT CRIM
    FROM boston
    ORDER BY CRIM
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2    -- odd 1, even 2
    OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))) = MIN(CRIM) THEN 'CRIM'
    ELSE ''
    END || ' ' ||
    
    CASE
    WHEN (SELECT AVG(ZN)
    FROM (SELECT ZN
    FROM boston
    ORDER BY ZN
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2    -- odd 1, even 2
    OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))) = MIN(ZN) THEN 'ZN'
    ELSE ''
    END || ' ' ||
    
    CASE
    WHEN (SELECT AVG(INDUS)
    FROM (SELECT INDUS
    FROM boston
    ORDER BY INDUS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2    -- odd 1, even 2
    OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))) = MIN(INDUS) THEN 'INDUS'
    ELSE ''
    END || ' ' ||
    
    CASE
    WHEN (SELECT AVG(CHAS)
    FROM (SELECT CHAS
    FROM boston
    ORDER BY CHAS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2    -- odd 1, even 2
    OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))) = MIN(CHAS) THEN 'CHAS'
    ELSE ''
    END || ' ' ||
    
    CASE
    WHEN (SELECT AVG(NOX)
    FROM (SELECT NOX
    FROM boston
    ORDER BY NOX
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2    -- odd 1, even 2
    OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))) = MIN(NOX) THEN 'NOX'
    ELSE ''
    END as 'Columns'
    
    FROM boston
    """,
    con,
)

,Columns
0,ZN CHAS


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [95]:
pd.read_sql(
    """
    SELECT AVG(RM) as avg_rm, MAX(MEDV) as mincount
    FROM boston
    UNION
    SELECT AVG(one.RM), "min25"
    FROM
    (SELECT * FROM boston ORDER BY MEDV ASC limit 25) one
    UNION
    SELECT AVG(one.RM), "min50"
    FROM
    (SELECT * FROM boston ORDER BY MEDV ASC limit 50) one
    UNION
    SELECT AVG(one.RM), "min100"
    FROM
    (SELECT * FROM boston ORDER BY MEDV ASC limit 100) one
    UNION
    SELECT AVG(one.RM), "min200"
    FROM
    (SELECT * FROM boston ORDER BY MEDV ASC limit 200) one
    UNION
    SELECT AVG(one.RM), "min300"
    FROM
    (SELECT * FROM boston ORDER BY MEDV ASC limit 300) one
    """,
    con,
)

,avg_rm,mincount
0,5.747840,min25
1,5.753240,min50
2,5.887120,min100
3,5.911705,min200
4,5.972227,min300
5,6.284634,50.0


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [123]:
pd.read_sql(
    """
    SELECT LSTAT, MEDV, RANK() OVER(ORDER BY LSTAT) FROM boston b
    --WHERE MEDV IN (SELECT MAX(MEDV) FROM boston)
    --limit 5
    """,
    con,
)

,LSTAT,MEDV,RANK() OVER(ORDER BY LSTAT)
0,1.73,50.0,1
1,1.92,50.0,2
2,1.98,34.9,3
3,2.47,41.7,4
4,2.87,36.4,5
...,...,...,...
501,34.37,17.9,502
502,34.41,14.4,503
503,34.77,13.8,504
504,36.98,7.0,505


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [133]:
pd.read_sql(
    """
    SELECT CHAS, AVG(MEDV)
    FROM boston
    GROUP BY CHAS
    --SELECT AVG(MEDV) OVER(PARTITION BY CHAS) as avg_, CHAS, MEDV
    --FROM boston
    --limit 5
    """,
    con,
)

,CHAS,AVG(MEDV)
0,0.0,22.093843
1,1.0,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [139]:
pd.read_sql(
    """
    SELECT
    DISTINCT
    CASE
    WHEN (SELECT AVG(CRIM) FROM boston) < (SELECT AVG(CRIM) FROM BOSTON WHERE CHAS = 1) THEN 'CRIM'
    ELSE '' 
    END
    || ' ' ||
    CASE
    WHEN (SELECT AVG(ZN) FROM boston) < (SELECT AVG(ZN) FROM BOSTON WHERE CHAS = 1) THEN 'ZN'
    ELSE ''
    END 
    || ' ' ||
    CASE
    WHEN (SELECT AVG(INDUS) FROM boston) < (SELECT AVG(INDUS) FROM BOSTON WHERE CHAS = 1) THEN 'INDUS'
    ELSE ''
    END 
    || ' ' ||
    CASE
    WHEN (SELECT AVG(CHAS) FROM boston) < (SELECT AVG(CHAS) FROM BOSTON WHERE CHAS = 1) THEN 'CHAS'
    ELSE ''
    END 
    || ' ' ||
    CASE
    WHEN (SELECT AVG(NOX) FROM boston) < (SELECT AVG(NOX) FROM BOSTON WHERE CHAS = 1) THEN 'NOX'
    ELSE ''
    END 
    
    as 'Columns'
    
    FROM boston
    
    """,
    con,
)

,Columns
0,INDUS CHAS NOX


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [232]:
pd.read_sql(
    """
    SELECT AVG(INDUS),AVG(NOX), b1.id FROM
    (SELECT 
    INDUS, '10per' as id
    FROM boston 
    ORDER BY INDUS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*) - 1 )/ 10 FROM boston) ) b1
    LEFT JOIN
    (SELECT 
    NOX, '10per' as id
    FROM boston 
    ORDER BY NOX
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*) - 1 )/ 10 FROM boston) ) b2
         
    UNION
    
    SELECT AVG(INDUS),AVG(NOX), b1.id FROM
    (SELECT 
    INDUS, '20per' as id
    FROM boston 
    ORDER BY INDUS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         ((COUNT(*) - 1 )/ 5) FROM boston) ) b1
    LEFT JOIN
    (SELECT 
    NOX, '20per' as id
    FROM boston 
    ORDER BY NOX
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         ((COUNT(*) - 1 )/ 5) FROM boston) ) b2
         
    UNION
    
    SELECT AVG(INDUS),AVG(NOX), b1.id FROM
    (SELECT 
    INDUS, '30per' as id
    FROM boston 
    ORDER BY INDUS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*3/10  FROM boston) ) b1
    LEFT JOIN
    (SELECT 
    NOX, '30per' as id
    FROM boston 
    ORDER BY NOX
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*3/10  FROM boston) ) b2
         
    UNION
    
    SELECT AVG(INDUS),AVG(NOX), b1.id FROM
    (SELECT 
    INDUS, '40per' as id
    FROM boston 
    ORDER BY INDUS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*4/10  FROM boston) ) b1
    LEFT JOIN
    (SELECT 
    NOX, '40per' as id
    FROM boston 
    ORDER BY NOX
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*4/10  FROM boston) ) b2
         
    UNION
    
    SELECT AVG(INDUS),AVG(NOX), b1.id FROM
    (SELECT 
    INDUS, '50per' as id
    FROM boston 
    ORDER BY INDUS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*5/10  FROM boston) ) b1
    LEFT JOIN
    (SELECT 
    NOX, '50per' as id
    FROM boston 
    ORDER BY NOX
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*5/10  FROM boston) ) b2
         
    UNION
    
    SELECT AVG(INDUS),AVG(NOX), b1.id FROM
    (SELECT 
    INDUS, '60per' as id
    FROM boston 
    ORDER BY INDUS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*6/10  FROM boston) ) b1
    LEFT JOIN
    (SELECT 
    NOX, '60per' as id
    FROM boston 
    ORDER BY NOX
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*6/10  FROM boston) ) b2
         
    UNION
    
    SELECT AVG(INDUS),AVG(NOX), b1.id FROM
    (SELECT 
    INDUS, '70per' as id
    FROM boston 
    ORDER BY INDUS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*7/10  FROM boston) ) b1
    LEFT JOIN
    (SELECT 
    NOX, '70per' as id
    FROM boston 
    ORDER BY NOX
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*7/10  FROM boston) ) b2
         
    UNION
    
    SELECT AVG(INDUS),AVG(NOX), b1.id FROM
    (SELECT 
    INDUS, '80per' as id
    FROM boston 
    ORDER BY INDUS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*8/10  FROM boston) ) b1
    LEFT JOIN
    (SELECT 
    NOX, '80per' as id
    FROM boston 
    ORDER BY NOX
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*8/10  FROM boston) ) b2
         
    UNION
    
    SELECT AVG(INDUS),AVG(NOX), b1.id FROM
    (SELECT 
    INDUS, '90per' as id
    FROM boston 
    ORDER BY INDUS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*9/10  FROM boston) ) b1
    LEFT JOIN
    (SELECT 
    NOX, '90per' as id
    FROM boston 
    ORDER BY NOX
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*9/10  FROM boston) ) b2
         
    UNION
    
    SELECT AVG(INDUS),AVG(NOX), b1.id FROM
    (SELECT 
    INDUS, '100per' as id
    FROM boston 
    ORDER BY INDUS
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*10/10  FROM boston) ) b1
    LEFT JOIN
    (SELECT 
    NOX, '100per' as id
    FROM boston 
    ORDER BY NOX
    LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
    OFFSET (SELECT
         (COUNT(*)-1 )*10/10  FROM boston) ) b2

   
    """,
    con,
)

,AVG(INDUS),AVG(NOX),id
0,2.91,0.42700,10per
1,4.39,0.44245,20per
2,5.96,0.47200,30per
3,7.38,0.50700,40per
4,9.69,0.53800,50per
5,12.83,0.57750,60per
6,18.10,0.60500,70per
7,18.10,0.66950,80per
8,19.58,0.71300,90per
9,27.74,0.87100,100per
